* Applying the Sigmoid Signal on TEC Token Holders
* Checking on and off signals

In [164]:
import param as pm
import panel as pn
import holoviews as hv
import pandas as pd
import numpy as np
import hvplot.pandas

In [165]:
df = pd.read_csv('output/TEGR1.csv')

In [210]:
class TECQFSME(pm.Parameterized):
    boost_factor = pm.Number(1, bounds=(0,4), step=0.1, doc="Multiplicative factor to apply to boosting coefficient.")
    dataset  = pm.DataFrame(columns={'amountUSD', 'projectId'}, precedence=-1, doc="Dataset of donations. Must contain amountUSD and projectId columns.")
    matching_pool = pm.Integer(25_000, bounds=(0, 100_000), step=5_000, doc="Matching pool amount.")
    total_donations = pm.Number(0, constant=True, doc="Summation of amountUSD from donations dataset.")
    total_funding_pool = pm.Number(0, constant=True, doc="Summation of matching_pool and total_donations.")
    allocations  = pm.DataFrame(precedence=-1, doc="Percentages allocation table.")
    results  = pm.DataFrame(precedence=-1, doc="Matched and unmatched funding amounts. Allocation percentages times funding amounts.")
    
    def __init__(self, **params):
        super().__init__(**params)
        self.dataset = self.qf(self.dataset)
        self.update()
        
    @staticmethod
    def qf(df, column_name='amountUSD', new_column_name='quadratic_amount'):
        """
        Takes a specefied column as the donations column. Applies the QF algorithm to produce a new specefied column and intermediate calculation columns.
        """
        df = df.copy(deep=True)
        df[f'{column_name}_allocation'] = df[column_name] / df[column_name].sum()
        df[f'sqrt({column_name})'] = np.sqrt(df[column_name])
        df[f'sum(sqrt({column_name}))'] = df.groupby('projectId')[f'sqrt({column_name})'].transform('sum')
        df[f'sq(sum(sqrt({column_name})))'] = df[f'sum(sqrt({column_name}))'].transform(lambda x: x**2)
        df[f'{new_column_name}_allocation'] = df[f'sq(sum(sqrt({column_name})))'] / df[f'sq(sum(sqrt({column_name})))'].sum()
        df[new_column_name] = df[column_name].sum() * df[f'{new_column_name}_allocation']
        return df
    
    @staticmethod
    def signal_boost_v1(df, boost_factor, boost_column='amountUSD', new_column_name='amount_boosted'):
        """
        Given a dataset and a specefied column, applies the flag boost algorithm.
        Requires that the dataset contain 'tec_token_flag' and 'tea_flag'.
        """
        df['coefficient'] = 1 + boost_factor * (df['tec_tokens_flag'].astype(int) | df['tea_flag'].astype(int))
        df[new_column_name] = df[boost_column] * df['coefficient']
        return df

    
    @pm.depends('boost_factor', 'matching_pool', watch=True)
    def update(self):
        # Update total donations and funding pool
        with pm.edit_constant(self):
            self.total_donations = self.dataset['amountUSD'].sum()
            self.total_funding_pool = self.matching_pool + self.total_donations
        
        with pm.parameterized.batch_call_watchers(self):
            # Generate and apply the signal boosting coefficient
            self.dataset = self.signal_boost_v1(self.dataset, self.boost_factor, boost_column='amountUSD', new_column_name='amount_boosted')

            # Compute the Boosted Allocation
            self.dataset = self.qf(self.dataset, column_name='amount_boosted', new_column_name='quadratic_amount_boosted')

            # Remove the intermediate steps
            self.dataset = self.dataset[self.dataset.columns[~self.dataset.columns.str.contains('sqrt')]]

            # Create an allocations table that contains allocation percentages grouped and summed by project. 
            allocation_columns = ['projectId'] + list(self.dataset.columns[self.dataset.columns.str.contains('allocation')])
            self.allocations = self.dataset[allocation_columns].groupby('projectId').sum()

            # Generate the unmatched results table by multiplying allocation percentages by total donations
            unmatched_results = self.total_donations * self.allocations
            
            # Generate the matched results table by multiplying allocation percentages by total donations plus matching pool
            matched_results = self.total_funding_pool * self.allocations
            
            # Merge matched and unmatched results
            self.results = unmatched_results.merge(matched_results, left_index=True, right_index=True, suffixes=('_unmatched', '_matched'))
            
            # Sort results by funding amount
            self.results = self.results.sort_values('quadratic_amount_allocation_matched', ascending=False)

            # Save the boosting percentage stat
            self.results['Percentage Boost'] = 100 * ((self.results['quadratic_amount_boosted_allocation_matched'] - self.results['quadratic_amount_allocation_matched']) / self.results['quadratic_amount_allocation_matched'] + 1)

        
    @pm.depends('dataset')
    def view_expertise_signal(self):
        return self.results.rename({
            'quadratic_amount_allocation_unmatched': 'QF Unmatched', 
            'quadratic_amount_allocation_matched': 'QF Matched', 
            'quadratic_amount_boosted_allocation_matched': 'QF Matched + SME',
        }, axis=1).hvplot.bar(
            y=['QF Unmatched', 'QF Matched', 'QF Matched + SME'],
            rot=45,
            stacked=False,
            title="Adding Expertise into the QF Signal",
        ).opts(multi_level=False, legend_position='top_right')
    
    @pm.depends('dataset')
    def view_percentage_boost(self):
        return self.results.hvplot.bar(
            y='Percentage Boost', 
            color='purple', 
            ylim=(0, 180), 
            yformatter="%.0f%%", 
            yticks=list(range(0,200,20)),
            grid=True,
            height=400,
            rot=45,
            title="SME Boost Funding as % of QF Funding by Project",
        ) * hv.HLine(100)
    
    @pm.depends('dataset')
    def view_sme_signal_boost_by_donation(self):
        return self.dataset.hvplot.area(
            y='amountUSD', y2='amount_boosted', 
            logy=True, color='green', title="Logarithmic SME Signal Boost by Donation", 
            xlabel='index')
            
    
    def view(self):
        return pn.Row(self.param, pn.Column(self.view_sme_signal_boost_by_donation, self.view_percentage_boost))

In [211]:
tec_qf_sme = TECQFSME(dataset=df.copy(deep=True))

In [212]:
tec_qf_sme.view()

Row
    [0] Column(margin=(5, 10), name='TECQFSME')
        [0] StaticText(value='<b>TECQFSME</b>')
        [1] FloatSlider(end=4, name='Boost factor', value=1)
        [2] IntSlider(end=100000, name='Matching pool', step=5000, value=25000)
        [3] FloatInput(description='Summation of a..., disabled=True, name='Total donations', value=6069.72067821)
        [4] FloatInput(description='Summation of m..., disabled=True, name='Total funding pool', value=31069.72067821)
    [1] Column
        [0] ParamMethod(method, _pane=HoloViews, defer_load=False)
        [1] ParamMethod(method, _pane=HoloViews, defer_load=False)

In [185]:
dataset = tec_qf_sme.dataset

In [186]:
dataset

,id,projectId,applicationId,voter,grantAddress,amount,amountUSD,coefficient,rawScore,balance_tec,tec_tokens_flag,balance_tea,tea_flag,amountUSD_allocation,quadratic_amount_allocation,quadratic_amount,amount_boosted,amount_boosted_allocation,quadratic_amount_boosted_allocation,quadratic_amount_boosted
0,0x24a5bbf1,0x64a30a4b,19,0x9ba96198,0xA26d6AEB,5.000000e+15,9.184332,5,28.57,0.0,0.0,3.0,1.0,0.001513,0.002959,17.961853,45.921662,0.002434,0.004119,77.714786
1,0x3dce13bb,0xc401c980,6,0x9390fa86,0x9390fA86,2.200000e+15,4.094567,1,27.21,0.0,0.0,0.0,0.0,0.000675,0.000185,1.124492,4.094567,0.000217,0.000146,2.763403
2,0x4cf20243,0x97589cd1,7,0x5136cdfc,0x0035cC37,4.000000e+16,74.446665,1,28.57,0.0,0.0,0.0,0.0,0.012265,0.002593,15.737962,74.446665,0.003946,0.002804,52.897193
3,0x2b032f10,0xec026845,16,0x524cb61b,0x45b79C6b,3.000000e+15,5.583500,1,23.56,0.0,0.0,0.0,0.0,0.000920,0.001863,11.309989,5.583500,0.000296,0.001943,36.655682
4,0x0842753b,0xa9bdf738,29,0x524cb61b,0x5041A1C1,3.000000e+15,5.583500,1,23.56,0.0,0.0,0.0,0.0,0.000920,0.003139,19.051858,5.583500,0.000296,0.003578,67.506757
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
248,0x26e1e300,0x97589cd1,7,0x4405f427,0x0035cC37,1.000000e+15,1.847803,1,29.74,0.0,0.0,0.0,0.0,0.000304,0.002593,15.737962,1.847803,0.000098,0.002804,52.897193
249,0xa21ca1aa,0xec026845,16,0xcdfbbe10,0x45b79C6b,1.000000e+15,1.843793,1,21.07,0.0,0.0,0.0,0.0,0.000304,0.001863,11.309989,1.843793,0.000098,0.001943,36.655682
250,0x634b5156,0xf1f4942d,24,0xcdfbbe10,0x4f8c531d,1.000000e+15,1.843793,1,21.07,0.0,0.0,0.0,0.0,0.000304,0.001476,8.957351,1.843793,0.000098,0.000964,18.193771
251,0x4efa29aa,0xcf3165f4,10,0x410d86e3,0x7f3eb18E,1.000000e+15,1.843793,1,18.04,0.0,0.0,0.0,0.0,0.000304,0.000050,0.301167,1.843793,0.000098,0.000035,0.664069


In [187]:
dataset.hvplot.area(y='amountUSD', y2='amount_boosted', logy=False, color='green', title="SME Signal Boost by Donation", width=1000, height=500, ylim=(0,None), xlabel='index')

:Area   [index]   (amountUSD,amount_boosted)

In [188]:
dataset.hvplot.area(y='amountUSD', y2='amount_boosted', logy=True, color='green', title="Logarithmic SME Signal Boost by Donation", width=1000, height=500, xlabel='index')

:Area   [index]   (amountUSD,amount_boosted)

In [189]:
dataset['Donation Index per Project'] = dataset.groupby('projectId').cumcount() + 1

In [190]:
dataset.hvplot.area(y='amountUSD', y2='amount_boosted',  by='projectId', x='Donation Index per Project', stacked=False, alpha=1, title="SME Signal Boost by Donation by Project", width=1000, height=500, ylim=(0,None)).opts(legend_position='top_right')#.opts(hv.opts.Area(color=hv.Palette('Category20')))

:NdOverlay   [projectId]
   :Area   [Donation Index per Project]   (amountUSD,amount_boosted)

In [213]:
dataset.hvplot.area(y='amountUSD', y2='amount_boosted', by='projectId', x='Donation Index per Project', stacked=False, alpha=1, logy=True, title="Logarithmic SME Signal Boost by Donation by Project", width=1000, height=500).opts(legend_position='top_right')#.opts(hv.opts.Area(color=hv.Palette('Category20')))

:NdOverlay   [projectId]
   :Area   [Donation Index per Project]   (amountUSD,amount_boosted)

In [192]:
dataset['Donation Index per Donor'] = dataset.groupby('voter').cumcount() + 1

In [193]:
dataset.hvplot.area(
    y='amountUSD', y2='amount_boosted',  by='voter', x='Donation Index per Donor', stacked=False, alpha=1, 
    title="SME Signal Boost by Donation by Donor",
    width=1000, height=500, ylim=(0,None)).opts(legend_position='top_right')#.opts(hv.opts.Area(color=hv.Palette('Category20')))

:NdOverlay   [voter]
   :Area   [Donation Index per Donor]   (amountUSD,amount_boosted)

In [194]:
dataset.hvplot.area(
    y='amountUSD', y2='amount_boosted',  by='voter', x='Donation Index per Donor', stacked=False, alpha=1, 
    title="Logarithmic SME Signal Boost by Donation by Donor", logy=True,
    width=1000, height=500, ylim=(0,None)).opts(legend_position='top_right')#.opts(hv.opts.Area(color=hv.Palette('Category20')))

:NdOverlay   [voter]
   :Area   [Donation Index per Donor]   (amountUSD,amount_boosted)

In [156]:
tec_qf_sme.allocations

,amountUSD_allocation,quadratic_amount_allocation,amount_boosted_allocation,quadratic_amount_boosted_allocation
projectId,,,,
0x10b3f00e,0.007163,0.001570,0.007283,0.001572
0x23387567,0.165409,0.037399,0.214523,0.047494
0x4cd41869,0.083889,0.177688,0.056623,0.127886
0x5351510d,0.008092,0.002346,0.008471,0.002443
0x64a30a4b,0.059451,0.050307,0.073838,0.059341
0x72b0d6a6,0.011525,0.004762,0.014014,0.005735
0x8d6f0c7b,0.037947,0.083346,0.043465,0.095413
0x97589cd1,0.045227,0.049264,0.046960,0.051512
0xa9bdf738,0.031684,0.078471,0.035111,0.084089


In [157]:
tec_qf_sme.view()

Row
    [0] Column(margin=(5, 10), name='TECQFSME')
        [0] StaticText(value='<b>TECQFSME</b>')
        [1] FloatSlider(end=2, name='Boost factor', value=1)
        [2] IntSlider(end=100000, name='Matching pool', step=5000, value=25000)
        [3] FloatInput(description='Summation of a..., disabled=True, name='Total donations', value=6069.72067821)
        [4] FloatInput(description='Summation of m..., disabled=True, name='Total funding pool', value=31069.72067821)
    [1] ParamMethod(method, _pane=HoloViews, defer_load=False)

In [158]:
tec_qf_sme.view_expertise_signal()

:Bars   [projectId,Variable]   (value)